In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [11]:
query = """
SELECT created_date, closed_date, descriptor,
    location_type, incident_zip,
    borough, latitude, longitude
FROM 
`bigquery-public-data.new_york.311_service_requests` 
WHERE complaint_type = 'HEAT/HOT WATER'
--LIMIT 1000

"""

In [12]:
df = pd.read_gbq(query, project_id='np-hacks',verbose=True, private_key='auth_key.json', dialect='standard')


Requesting query... ok.
Job ID: job_Y0Bq0zwhlu8RihpMnTcBJ4fMrkwD
Query running...
Query done.
Processed: 1.5 GB
Standard price: $0.01 USD

Retrieving results...
  Got page: 1; 6.0% done. Elapsed 14.81 s.
  Got page: 2; 12.0% done. Elapsed 21.16 s.
  Got page: 3; 19.0% done. Elapsed 28.26 s.
  Got page: 4; 25.0% done. Elapsed 33.21 s.
  Got page: 5; 31.0% done. Elapsed 39.53 s.
  Got page: 6; 37.0% done. Elapsed 46.1 s.
  Got page: 7; 44.0% done. Elapsed 51.8 s.
  Got page: 8; 50.0% done. Elapsed 59.52 s.
  Got page: 9; 56.0% done. Elapsed 65.76 s.
  Got page: 10; 62.0% done. Elapsed 71.3 s.
  Got page: 11; 68.0% done. Elapsed 79.14 s.
  Got page: 12; 75.0% done. Elapsed 85.26 s.
  Got page: 13; 81.0% done. Elapsed 91.15 s.
  Got page: 14; 87.0% done. Elapsed 100.41 s.
  Got page: 15; 93.0% done. Elapsed 106.96 s.
  Got page: 16; 100.0% done. Elapsed 112.95 s.
  Got page: 17; 100.0% done. Elapsed 113.87 s.
Got 703178 rows.

Total time taken 165.36 s.
Finished at 2017-09-23 13:23:42.


In [13]:
df.head()

,created_date,closed_date,descriptor,location_type,incident_zip,borough,latitude,longitude
0,2016-12-16 16:48:23,2016-12-26 02:06:47,APARTMENT ONLY,RESIDENTIAL BUILDING,11691,QUEENS,40.602244,-73.753957
1,2015-06-04 00:00:00,2015-06-09 00:00:00,ENTIRE BUILDING,RESIDENTIAL BUILDING,11691,QUEENS,40.602244,-73.753957
2,2016-10-25 15:09:49,2016-10-30 08:48:56,ENTIRE BUILDING,RESIDENTIAL BUILDING,11691,QUEENS,40.602244,-73.753957
3,2016-10-10 07:36:32,2016-10-12 10:56:01,ENTIRE BUILDING,RESIDENTIAL BUILDING,11691,QUEENS,40.602244,-73.753957
4,2014-03-31 00:00:00,2014-04-07 00:00:00,ENTIRE BUILDING,RESIDENTIAL BUILDING,11691,QUEENS,40.602244,-73.753957


In [14]:
df.to_csv("hhw.csv")

In [30]:
# per fiscal year, total calls and mean time to resolution
query2 = """
SELECT AVG(close_days) AS avg_close_days,
    count(*) AS n,
    fiscal_year
FROM
(
SELECT DATETIME_DIFF(DATETIME(closed_date), DATETIME(created_date), DAY) AS close_days, 
    EXTRACT(YEAR FROM DATETIME_ADD(DATETIME(created_date), INTERVAL 6 MONTH)) AS fiscal_year
--     created_date,
--     closed_date
FROM 
`bigquery-public-data.new_york.311_service_requests` 
WHERE complaint_type = 'HEAT/HOT WATER'
) x
GROUP BY fiscal_year
ORDER BY fiscal_year
LIMIT 1000

"""


In [31]:
df2 = pd.read_gbq(query2, project_id='np-hacks',verbose=True, private_key='auth_key.json', dialect='standard')

Requesting query... ok.
Job ID: job_2YzZpEla_xhvH24RVjTMdTdEt50k
Query running...
Query done.
Processed: 513.6 MB
Standard price: $0.00 USD

Retrieving results...
Got 5 rows.

Total time taken 2.03 s.
Finished at 2017-09-23 13:35:13.


In [32]:
df2

,avg_close_days,n,fiscal_year
0,3.337840,34853,2014
1,3.739793,240710,2015
2,3.541129,210362,2016
3,3.034665,210422,2017
4,4.135277,6831,2018


In [33]:
df2.to_csv("hhw_summary.csv")